In [1]:
!pip install selenium

In [2]:
!pip install bs4

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
import time

In [4]:
PATH = "./chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(PATH),chrome_options=chrome_options)

C:\Users\DELL\AppData\Local\Temp\ipykernel_9936\3965837803.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(PATH),chrome_options=chrome_options)


In [5]:
from bs4 import BeautifulSoup
def link_from_category_parallel(category_link, category, n_pages):
    
  class_from_categ = {"News":"SoaBEf", "Videos":"MjjYud"} #class tag for categories
  class_tag = ""
  class_tag = class_from_categ[category]

  results = [] # list for storing all the links


  for page in range(1, n_pages):
    url = category_link +  str((page - 1) * 10) 
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    search = soup.find_all('div', class_=class_tag )
    for h in search:
        results.append(h.a.get('href'))

In [6]:
import threading

def links_for_search_parallel(query, n_pages=10):

  # redirecting to google.com
  driver.get("https://www.google.com")

  # accessing the search bar and searching the specified query
  search_bar = driver.find_element("name", "q")
  search_bar.clear()
  search_bar.send_keys(query)
  search_bar.send_keys(Keys.RETURN)

  # fetching the news and videos links for the specified query
  category_list = ["News", "Videos"]
  category_link = []
  for i in category_list:
    category_link.append(driver.find_element(By.LINK_TEXT, i).get_attribute('href'))
    
  result_links = []

  # PARALLELISING THE SEARCH RESULTS FOR VIDEOS AND NEWS ARTICLES

  # creating a thread for fetching all the links for news articles
  t1 = threading.Thread(target=link_from_category_parallel, args=(category_link[0], "News",n_pages))

  # creating a thread for fetching all the links for videos
  t2 = threading.Thread(target=link_from_category_parallel, args=(category_link[1], "Videos",n_pages))

  # starting thread 1
  t1.start()
  # starting thread 2
  t2.start()

  # wait until thread 1 is completely executed
  t1.join()
  # wait until thread 2 is completely executed
  t2.join()

  print("Done!!")

In [8]:
search_req = "Data Science"

parallel_start_time = time.time()
links = links_for_search_parallel(search_req,10) # links_for_search(your_query, no_of_pages)
parallel_end_time = time.time() - parallel_start_time

print("Time taken for parallel implementation:", parallel_end_time)

Done!!
Time taken for parallel implementation: 12.766128063201904
